In [1]:
!pip install git+https://github.com/cayleypy/cayleypy -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.

In [2]:
import pandas as pd

test = pd.read_csv("/kaggle/input/CayleyPy-pancake/test.csv")
test["n"].unique()

array([  5,  12,  15,  16,  20,  25,  30,  35,  40,  45,  50,  75, 100])

In [3]:
def pancake_sort_path(perm: list[int]) -> list[str]:
    """Return a sequence of prefix reversals that sorts `perm` to the identity permutation."""
    arr = list(perm)
    n = len(arr)
    moves: list[str] = []

    for target in range(n, 1, -1):
        desired_value = target - 1
        idx = arr.index(desired_value)

        if idx == target - 1:
            continue  # already in place

        if idx != 0:
            moves.append(f'R{idx + 1}')
            arr[: idx + 1] = reversed(arr[: idx + 1])

        moves.append(f'R{target}')
        arr[:target] = reversed(arr[:target])

    return moves

In [4]:
import torch
from torch import nn

class StateScorer(nn.Module):
    def __init__(self, n_pancakes: int, hidden_dim: int):
        super().__init__()
        self.n_pancakes = n_pancakes
        self.net = nn.Sequential(nn.Linear(n_pancakes, hidden_dim), nn.LeakyReLU(), nn.Linear(hidden_dim, 1))

    def forward(self, x):
        return self.net(2*x.float() / (self.n_pancakes - 1) - 1).squeeze(-1)

In [5]:
import numpy as np
import random

def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.use_deterministic_algorithms(True)

In [20]:
from cayleypy import CayleyGraph, PermutationGroups
from cayleypy.algo import RandomWalksGenerator
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

def train_predictor(n, graph=None, width=1000, length=5, n_dim=100, batch_size=16, epochs=8, random_seed=42):
    seed_everything(random_seed)
    if graph is None:
        graph = CayleyGraph(PermutationGroups.pancake(n))
    rwg = RandomWalksGenerator(graph)
    states, distances = rwg.generate(width=width, length=length, mode="classic")
    train_dataset = TensorDataset(states.float(), distances.float())
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
    predictor = StateScorer(n, n_dim)
    predictor.train()

    optimizator = torch.optim.AdamW(predictor.parameters(), lr=3e-4)
    loss_fn = nn.MSELoss()

    for _ in range(epochs):
        for states, dist in tqdm(train_dataloader):
            optimizator.zero_grad()
            pred_dist = predictor(states)
            loss = loss_fn(pred_dist, dist)
            loss.backward()
            optimizator.step()

    predictor.eval()
    return predictor

# Abilation

In [16]:
n = 10
graph = CayleyGraph(PermutationGroups.pancake(n))

In [17]:
seed_everything(0)
test_start_states = [np.random.permutation(n) for _ in range(100)]
heurestic_path = [pancake_sort_path(x) for x in test_start_states]

def measure_success(predictor, beam_width):
    success_count = 0
    sum_length = 0
    diff_length = 0
    for state, hp in tqdm(zip(test_start_states, heurestic_path), total=len(test_start_states)):
        graph.free_memory()
        result = graph.beam_search(start_state=state, beam_width=beam_width, max_steps=n*2, predictor=predictor)
        if result.path_found:
            success_count += 1
            sum_length += result.path_length
            diff_length += result.path_length - len(hp)
    return success_count/len(test_start_states), sum_length/max(success_count, 1), diff_length/max(success_count, 1)

In [19]:
from cayleypy import Predictor
from itertools import product

seed_everything(42)
i = 1
max_i = 4*4*3
epochs = 8
n_dim = 1000
results = []
for width in [5, 10, 100, 1000]:
    for length in [5, 10, int(n*1.67), 2*n]:
        for n_dim in [10, 100, 1000]:
             result = {}
             result["predictor"] = Predictor(graph, train_predictor(n, graph=graph, width=width, length=length, n_dim=n_dim, epochs=epochs))
             hit_rate, avg_leng, avg_diff = measure_success(result["predictor"], 10)
             result["hit_rate"] = hit_rate
             result["avg_leng"] = avg_leng
             result["avg_diff"] = avg_diff
             result["width"] = width
             result["length"] = length
             result["n_dim"] = n_dim
             result["epochs"] = epochs
             results.append(result)
             print(f"{i}/{max_i}")
             i += 1

100%|██████████| 100/100 [00:31<00:00,  3.19it/s]


1/48


100%|██████████| 100/100 [00:31<00:00,  3.17it/s]


2/48


100%|██████████| 100/100 [00:32<00:00,  3.04it/s]


...

46/48


100%|██████████| 100/100 [00:26<00:00,  3.76it/s]


47/48


100%|██████████| 100/100 [00:25<00:00,  3.89it/s]

48/48


In [ ]:
import numpy as np
from cayleypy import Predictor
from tqdm import tqdm

test_start_states = [np.random.permutation(n_pancakes) for _ in range(100)]

beam_size_range = [1, 10, 10**2]
hamming_success_rates = []
nn_success_rates = []
for beam_size in beam_size_range:
    hamming_success_rates.append(measure_success(Predictor(graph, "hamming"), beam_size))
    nn_success_rates.append(measure_success(Predictor(graph, scorer), beam_size))

In [ ]:
from matplotlib import pyplot as plt

plt.plot(beam_size_range, hamming_success_rates, label="Hamming distance", marker='.')
plt.plot(beam_size_range, nn_success_rates, label="Neural network", marker='.')
plt.xlabel('Beam size')
plt.ylabel('Success rate')
plt.legend()
plt.xscale('log')
plt.title("Beam search for LRX(12)")
plt.show()

# Prediction

In [23]:
import numpy as np
from cayleypy import Predictor
from tqdm import tqdm

graphs = {}
models = {}
for n_s in test["n"].unique():
    graphs[n_s] = CayleyGraph(PermutationGroups.pancake(n_s))
    models[n_s] = Predictor(graphs[n_s], train_predictor(n_s, graph=graphs[n_s], width=n_s*100, length=int(n_s*1.6), batch_size=32, epochs=4))

100%|██████████| 50000/50000 [01:21<00:00, 612.08it/s]


In [9]:
def pancake_sort_path(perm: list[int]) -> list[str]:
    """Return a sequence of prefix reversals that sorts `perm` to the identity permutation."""
    arr = list(perm)
    n = len(arr)
    moves: list[str] = []

    for target in range(n, 1, -1):
        desired_value = target - 1
        idx = arr.index(desired_value)

        if idx == target - 1:
            continue  # already in place

        if idx != 0:
            moves.append(f'R{idx + 1}')
            arr[: idx + 1] = reversed(arr[: idx + 1])

        moves.append(f'R{target}')
        arr[:target] = reversed(arr[:target])

    return moves

In [10]:
import numpy as np

heurestic_paths = []
for _, row in tqdm(test.iterrows(), total=len(test)):
    perms = np.array(row["permutation"].split(",")).astype(int)
    moves = pancake_sort_path(perms)
    heurestic_paths.append(".".join(moves))

100%|██████████| 2405/2405 [00:00<00:00, 5771.70it/s] 


In [31]:
pred_paths = []

for i, row in tqdm(test.iterrows(), total=len(test)):
    n = row["n"]
    if n >= 20:
        pred_paths.append(heurestic_paths[i])
        continue
    heurestic_length = heurestic_paths[i].count(".") + 1
    perms = np.array(row["permutation"].split(",")).astype(int)
    graphs[n].free_memory()
    result = graphs[n].beam_search(start_state=perms, beam_width=1000, max_steps=heurestic_length, predictor=models[n], return_path=True)
    if result.path_found and len(result.path) < heurestic_length:
        pred_paths.append(".".join([f"R{gen_index+2}" for gen_index in result.path]))
    else:
        pred_paths.append(heurestic_paths[i])

100%|██████████| 2405/2405 [06:16<00:00,  6.39it/s]


In [32]:
submissions = pd.read_csv("/kaggle/input/CayleyPy-pancake/sample_submission.csv")
submissions["solution"] = pred_paths
submissions.to_csv("nn_submission.csv")